In [1]:
import numpy as np
import pandas as pd

In [2]:
import json

In [126]:
from sklearn.base import TransformerMixin
from sklearn_pandas import DataFrameMapper
from sklearn_pandas import CategoricalImputer
from sklearn.preprocessing import Imputer

In [3]:
ames = pd.read_csv("data/train.csv")
ames.head()

,Id,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


---

Right now, four strategies of interest to impute missing values:
* mean
* median
* mode
* custom (where missing values indicates some specific category like absence)
---
In absence of custom imputation strategy, variable will be imputed based on data type and distribution. Continuous variables should be imputed with Mean or Median, and Ordinal and Nominal with Mode. Since almost all continuous variables are skewed (mostly highly), Median is preferred over Mean.

In [4]:
with open('feature_details.json','r') as f:
    feat_d = json.load(f)

In [5]:
feat_d.keys()

dict_keys(['droplist', 'continuous', 'nominal', 'ordinal', 'impute_custom', 'impute_mode', 'impute_median'])

In [10]:
# variables whose missing values should be filled with custom strategy. 
# Where missing value means some predefined category or value.
impute_custom = ['Alley','BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2','FireplaceQu','GarageType']
impute_custom += ['GarageYrBlt','GarageFinish','GarageQual','GarageCond','PoolQC','Fence','MiscFeature']

In [17]:
# remove droplist variables from impute_custom
feat_d['impute_custom'] = [x for x in impute_custom if x not in feat_d['droplist']]

In [20]:
# all nominal and ordinal variables not in impute_custom will be imputed with mode, so put these in impute_mode.
# nominal and ordinal list was created after filtering droplist variables.
feat_d['impute_mode'] = [x for x in (feat_d['nominal']+feat_d['ordinal']) if x not in feat_d['impute_custom']]

In [23]:
# all continuous variables will be imputed with Median. Exclude those in impute_custom
# continuous list was also created after filtering droplist variables.
feat_d['impute_median'] = [x for x in feat_d['continuous'] if x not in feat_d['impute_custom']]

In [24]:
len(feat_d['impute_custom']) + len(feat_d['impute_mode']) + len(feat_d['impute_median']) + len(feat_d['droplist'])

80

In [25]:
with open('feature_details.json','w') as f:
    json.dump(feat_d, f)

In [138]:
class BsmtImputer(TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.fillna("NoBasement")

class GarageImputer(TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.fillna("NoGarage")

class AlleyImputer(TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.fillna("NoAlley")
    
class GarageYrImputer(TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.fillna(0)

class FireplaceQuImputer(TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.fillna("NoFireplace")
    
class FenceImputer(TransformerMixin):
    def fit(self, X, y=None):
        return self
    
    def transform(self, X):
        return X.fillna("NoFence")

In [134]:
def impute_ames():
    with open('feature_details.json','r') as f:
        feat_d = json.load(f)
    
    impute_defs = [([impute_median_col], Imputer(strategy='median')) for impute_median_col in feat_d['impute_median']]
    impute_defs.extend([(impute_mode_col, CategoricalImputer()) for impute_mode_col in feat_d['impute_mode']])
    # custom imputes
    impute_defs.extend([(bsmt_col, BsmtImputer()) for bsmt_col in ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']])
    impute_defs.extend([(garage_col, GarageImputer()) for garage_col in ['GarageType','GarageFinish','GarageQual','GarageCond']])
    impute_defs.append(('Alley', AlleyImputer()))
    impute_defs.append(('FireplaceQu', FireplaceQuImputer()))
    impute_defs.append(('GarageYrBlt', GarageYrImputer()))
    impute_defs.append(('Fence', FenceImputer()))
    
    return DataFrameMapper(impute_defs, input_df=True, df_out=True)
    

In [135]:
mpp = impute_ames()
a3 = mpp.fit_transform(ames.copy())

In [137]:
a3.isnull().sum()

LotFrontage      0
LotArea          0
YearBuilt        0
YearRemodAdd     0
MasVnrArea       0
BsmtFinSF1       0
BsmtUnfSF        0
TotalBsmtSF      0
1stFlrSF         0
2ndFlrSF         0
GrLivArea        0
TotRmsAbvGrd     0
GarageArea       0
WoodDeckSF       0
OpenPorchSF      0
EnclosedPorch    0
MSZoning         0
LotShape         0
LandContour      0
LotConfig        0
Neighborhood     0
Condition1       0
Condition2       0
BldgType         0
HouseStyle       0
RoofStyle        0
RoofMatl         0
Exterior1st      0
Exterior2nd      0
MasVnrType       0
                ..
Electrical       0
Functional       0
PavedDrive       0
SaleType         0
SaleCondition    0
MSSubClass       0
HeatingQC        0
KitchenQual      0
OverallQual      0
OverallCond      0
BsmtFullBath     0
FullBath         0
HalfBath         0
BedroomAbvGr     0
KitchenAbvGr     0
Fireplaces       0
GarageCars       0
BsmtQual         0
BsmtCond         0
BsmtExposure     0
BsmtFinType1     0
BsmtFinType2

In [118]:
impute_defs = [([impute_median_col], Imputer(strategy='median')) for impute_median_col in feat_d['impute_median']]
impute_defs.extend([(impute_mode_col, CategoricalImputer()) for impute_mode_col in feat_d['impute_mode']])
# custom imputes
impute_defs.extend([(bsmt_col, BsmtImputer()) for bsmt_col in ['BsmtQual','BsmtCond','BsmtExposure','BsmtFinType1','BsmtFinType2']])
impute_defs.extend([(garage_col, GarageImputer()) for garage_col in ['GarageType','GarageFinish','GarageQual','GarageCond']])
impute_defs.append(('Alley', AlleyImputer()))
impute_defs.append(('FireplaceQu', FireplaceQuImputer()))
impute_defs.append(('GarageYrBlt', GarageYrImputer()))
impute_defs.append(('Fence', FenceImputer()))

mapper = DataFrameMapper(impute_defs, input_df=True, df_out=True)

In [119]:
ames2 = ames.copy()
aa = mapper.fit_transform(ames2)